# Enzyme analysis

1. Convert the enzyme variant to amino acids based on the AAs and be warned about 1 vs 0 so need to check
2. Calculate concentration rather than yield (if yield is given then yield * mmol concentration)
3. Normalize to parent for each column (TTN, Yield)
4. Embeddings sequences
5. Convert to LevSeq format

In [1]:
! export CUDA_VISIBLE_DEVICES=1

In [7]:
import pandas as pd
import os
import torch 

# CUDA setup
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")

data_dir = 'output'
files = os.listdir('Final CSV/')
all_dfs = pd.DataFrame()
for f in files:
    if '.csv' in f:
        df = pd.read_csv(f'Final CSV/{f}')
        df['paper'] = f
        all_dfs = pd.concat([all_dfs, df])
df = all_dfs


In [12]:
parents = df.drop_duplicates(subset='paper', keep='first')
parents['smiles_reaction'] = [s.replace(' ', '') for s in parents['smiles_reaction'].values]
parents.to_csv('output/parents_LLM_extraction.csv', index=False)

/tmp/ipykernel_2080899/1255963081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parents['smiles_reaction'] = [s.replace(' ', '') for s in parents['smiles_reaction'].values]


In [18]:
parents['intermediate'] = ['.'.join(s.split('>>')[0].split('.')[1:]) for s in parents['smiles_reaction'].values]
parents['substrate'] = [s.split('>>')[0].split('.')[0] for s in parents['smiles_reaction'].values]

/tmp/ipykernel_2080899/217822098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parents['intermediate'] = ['.'.join(s.split('>>')[0].split('.')[1:]) for s in parents['smiles_reaction'].values]
/tmp/ipykernel_2080899/217822098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parents['substrate'] = [s.split('>>')[0].split('.')[0] for s in parents['smiles_reaction'].values]


In [19]:
parents

,id,barcode_plate,plate,well,smiles_string,smiles_reaction,alignment_count,alignment_probability,nucleotide_mutation,amino_acid_substitutions,...,reaction_condition,campaign_id,generation,parent_enzyme_id,additional_information,paper,conversion,activity,intermediate,substrate
0,CarH*,1,Plate_1,A01,CCOC(=O)CC=Cc1ccc(OC)cc1.CCOC(=O)CCCc1ccc(OC)cc1,COc1ccc(C=C)cc1.CCOC(=O)C=[N+]=[N-]>>CCOC(=O)C...,1,1.0,NaN,NaN,...,reaction_temperature:4 °C;reaction_ph:7.5;reac...,CarH_Mutant_Screening,0.0,NaN,"{""x_axis_label"":""Parent CarH*"",""notes"":""X-axis...",dna-Controlling Non-Native Cobalamin Reactivit...,NaN,NaN,CCOC(=O)C=[N+]=[N-],COc1ccc(C=C)cc1
0,hCAII,1,Plate_1,A01,C[C@H](O)c1ccccc1,C1(=CC=CC=C1)C(C)=O.C1(=CC=CC=C1)[SiH3]>>C1(=C...,1,1.0,NaN,NaN,...,reaction_temperature:20 °C;reaction_ph:8.0;rea...,hCAII_active_site_mutant_characterization,0.0,NaN,"{""x_axis_label"":""wt-hCAII (0.05 mol%)"",""cataly...",dna-Abiotic reduction of ketones with silanes ...,NaN,NaN,C1(=CC=CC=C1)[SiH3],C1(=CC=CC=C1)C(C)=O
0,4M,1,Plate_1,A01,CCC(O)C(C)=O,O=C(C(=O)[O-])C.[Na+].C(CC)=O>>OC(C(C)=O)CC,1,1.0,NaN,NaN,...,reaction_temperature:30 °C;reaction_ph:7.0;rea...,TK_Pyruvate_Aldehyde_Evolution,0.0,NaN,"{""x_axis_label"":""4M"",""catalyst_form"":""purified...",dna-Engineering transketolase to accept both u...,NaN,NaN,[Na+].C(CC)=O,O=C(C(=O)[O-])C
0,22 C Sav K121A,1,Plate_1,B08,N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O,>>N[C@@H](CC1=CC(I)=C(C(I)=C1)OC1=CC(I)=C(C=C1...,1,1.0,A361G_A362C_A363T,K121A,...,reaction_temperature:37 °C;reaction_ph:7.5;rea...,Sav_Scaffold_Optimization_Screen,1.0,22 C Sav WT,"{""x_axis_label"":""A"",""conversion"":""nan"",""cataly...",A cell-penetrating artificial metalloenzyme re...,NaN,NaN,,
0,Mb (WT),1,Plate_1,A01,CCOC(=O)C1C2Oc3ccccc3C21,O1C=CC2=C1C=CC=C2.[N+](=[N-])=CC(=O)OCC>>C1(C2...,1,1.0,NaN,NaN,...,reaction_temperature:room temperature;reaction...,Mb_Benzofuran_Cyclopropanation_Engineering,0.0,NaN,"{""x_axis_label"":""Mb"",""catalyst_form"":""purified...",Biocatalytic Strategy for Highly Diastereo- an...,NaN,NaN,[N+](=[N-])=CC(=O)OCC,O1C=CC2=C1C=CC=C2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,LmrR_L57C_THP,1,Plate_1,A01,O=C1CCCCC1C(C[N+](=O)[O-])c1ccccc1,C1(CCCCC1)=O.[N+](=O)([O-])/C=C/C1=CC=CC=C1>>[...,1,1.0,NaN,NaN,...,reaction_temperature:20°C;reaction_ph:7.0;reac...,LmrR_Michael_Addition_Evolution,0.0,NaN,"{""x_axis_label"":""LmrR_L57C_THP"",""catalyst_form...",Design and Evolution of an Enzyme for the Asym...,NaN,NaN,[N+](=O)([O-])/C=C/C1=CC=CC=C1,C1(CCCCC1)=O
0,Rma TDE,1,Plate_1,A01,CCOC(=O)C(C)[Si](C)(C)c1ccccc1,CCOC(=O)C(C)=[N+]=[N-].C[SiH](C)c1ccccc1>>CCOC...,1,1.0,NaN,NaN,...,reaction_temperature:room temperature;reaction...,Rma_TDE_CarbeneTransferase_Characterization,1.0,Rma cyt c WT,"{""x_axis_label"":""Rma TDE"",""notes"":""TTN value o...",Catalytic iron-carbene intermediate revealed i...,NaN,NaN,C[SiH](C)c1ccccc1,CCOC(=O)C(C)=[N+]=[N-]
0,T53L/K210H RA95.5-8,1,Plate_1,A01,COc1ccc(C(CC(C)=O)C[N+](=O)[O-])cc1,COC1=CC=C(C=C1)\C=C\[N+](=O)[O-].CC(C)=O>>COC1...,1,1.0,NaN,T53L_K210H,...,reaction_temperature:29 °C;reaction_ph:6.5;rea...,default_characterization,1.0,RA95.5-8,"{""x_axis_label"":""T53L/K210H"",""catalyst_form"":""...",dna-Enantiocomplementary Synthesis of γ-Nitrok...,NaN,NaN,CC(C)=O,COC1=CC=C(C=C1)\C=C\[N+](=O)[O-]
0,CNH-0,1,Plate_1,A01,CCOC(=O)[C@H](C)N(C)c1ccccc1,CNc1ccccc1.CCOC(=O)C(C)=[N+]=[N-]>>CCOC(=O)[C@...,1,1.0,NaN,NaN,...,reaction_temperature:30 °C;reaction_ph:6.0;rea...,IrCYP119_NH_insertion_evolution,0.0,NaN,"{""x_axis_label"":""155F,213G,254L,317G (CNH-0)"",...",dna-Directed Evolution of Artificial Metalloen...,NaN,NaN,CCOC(=O)C(C)=[N+]=[N-],CNc1ccccc1


# Iterate through the parent amino acid sequence and make the variant sequence 

Since sometimes people use 0 index and sometimes 1 index, first check the parent AA sequnece has the correct AA on the LHS.

In [51]:
from sciutil import SciUtil

u = SciUtil()

def convert_to_variant(parent, aa_to_change, verbose=False):
    warn = []
    error = []
    variant = []
    #try:
    aa_to_change = aa_to_change.replace(' ', '').strip()
    variant = list(parent)
    parent = list(parent)
    sep = '_'
    # check what separator was used
    if '_' in aa_to_change:
        sep = '_'
    elif ',' in aa_to_change:
        sep = ','
    for aa in aa_to_change.split(sep):
        if 'FAD' not in aa:
            aa = aa.split('+')[0] # Remove any random domains... 
            try:
                parent_aa = aa[0]
                variant_aa = aa[-1]
                position = int(aa[1:-1])
                # Check the parent at the position is correct
                if position < len(parent):
                    if parent[position] == parent_aa and parent[position - 1] == parent_aa : # Can't distinguish... since they are the same!
                        if verbose:
                            u.warn_p(['Defaulting to 1 index since could not distinguish... warning to check sequence'])
                        variant[position + 1] = variant_aa
                        warn.append([aa, 'Warning: position 0 and 1 index had same AA used 1 index'])
                    elif parent[position] == parent_aa: # 0 index
                        variant[position] = variant_aa
                    elif parent[position - 1] == parent_aa: # 1 index
                        variant[position - 1] = variant_aa
                    else:
                        error.append([aa, 'Error in parent AA != AA in seq']) # Failed at this one we have an idea about where it was
                elif parent[position - 1] == parent_aa: # 1 index
                        variant[position - 1] = variant_aa
                else:
                    error.append([aa, 'Error in parent AA != AA in seq']) # Failed at this one we have an idea about where it was
            except Exception as e:
                error.append([aa, str(e)])
        else:
            u.warn_p(['FAD domain, manually check'])

    return ''.join(variant), warn, error
    
# Check these are the same since this is what we need
assert convert_to_variant('MTAKEMPQPKTFGELKNLPLLNTD', 'A2K')[0] == 'MTKKEMPQPKTFGELKNLPLLNTD' # 0 indexed example
assert convert_to_variant('MATKEMPQPKTFGELKNLPLLNTD', 'A2K')[0] == 'MKTKEMPQPKTFGELKNLPLLNTD' # 1 indexed
assert len(convert_to_variant('MAAKEMPQPKTFGELKNLPLLNTD', 'A2K')[1]) > 0 # Should print an error too 

In [52]:
from tqdm import tqdm 
from Bio.Seq import translate
import re

aas = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

reg = f"^[{''.join(aas)}]*$"

parent_aa = []
variant_aa = []
errors, warnings = [], []
changed = 0

for parent, parent_nt, aa_to_change, variant_nt in tqdm(df[['parent_aminoacid_sequence', 'parent_DNA_sequence', 'aminoacid_mutations_from_parent', 'variant_DNA_sequence']].values):
    if not parent or not isinstance(parent, str) or len(parent) < 2:
        # Try converting the parent nt
        if not parent_nt or not isinstance(parent_nt, str) or len(parent_nt) < 2:
            parent = None
            errors.append(['Parent was not string?'])
            warnings.append(['Parent was not string?'])
            parent_aa.append(None)
            continue
        parent_nt = parent_nt.replace(' ', '')
        parent = translate(parent_nt)
    parent = parent.replace(' ', '')
    parent = parent.strip()
    # We're replace his tags... 
    parent = parent.replace('LEHHHHHH', '').replace('HHHHHH', '').replace('*', '')
    # Check if the parent has any odd AAs
    if not re.match(reg, parent):
        # Parent has strange seqs..
        for ai, a in enumerate(parent):
            if a not in aas:
                print(a, ai)
        errors.append(['Parent had a non-correct AA in it.. check for numbers!'])
        variant_aa.append(None)
        warnings.append(None)
        parent_aa.append(parent)
        continue
    aa_to_change = aa_to_change.strip()
    if aa_to_change != '?' and parent is not None:
        variant, warn, err = convert_to_variant(parent, aa_to_change)
        # Check if it got truncated 
        variant = variant.split('*')[0] # Truncate 
        variant_aa.append(variant)
        # Ensure the variant only has amino acids
        if not re.match(reg, variant):
            # Parent has strange seqs..
            print('NO MATCH')
            break
        changed += 1
        if variant_aa == parent:
            print('Same?', aa_to_change)
        if len(warn) == 0: 
            warnings.append(None)
        else:
            warnings.append(warn)
        if len(err) == 0:
            errors.append(None)
        else:
            errors.append(err)
    else:
        variant_aa.append(None)
        errors.append(None)
        warnings.append(None)
    parent_aa.append(parent)


  0%|                                                             | 0/1341 [00:00<?, ?it/s]/disk1/ariane/miniconda3/envs/protein/lib/python3.11/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
100%|███████████████████████████████████████████████| 1341/1341 [00:00<00:00, 14469.52it/s]

--------------------------------------------------------------------------------
                          FAD domain, manually check	                           
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                          FAD domain, manually check	                           
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                          FAD domain, manually check	                           
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                          FAD domain, manually check	                           
--------------------------------------------------------------------------------
----------------------------

In [53]:
df['parent_aa'] = parent_aa
df['variant_aa'] = variant_aa
df['errors'] = errors
df['warnings'] = warnings
# Print out the number of errors
err_df = df[~df['errors'].isna()]
err_df['first author'].value_counts()

Juner Zhang    16
Name: first author, dtype: int64

# Continue with data that has no errors

Ensure that the required libraries are installed:

Install `enzyme-tk`


In [8]:
#! pip install enzymetk

# Create ESM2 or ESM3 embeddings of each protein



In [54]:
# Do ESM embedding of the ones that look good!
import sys
sys.path.append('/disk1/ariane/vscode/enzyme-tk')
from enzymetk.embedprotein_esm_step import EmbedESM
from enzymetk.save_step import Save
import pandas as pd

df['id'] = [f'DEDB{i}' for i in range(0, len(df))]
df = df[df['errors'].isna()]
# Fill in variants with parent if there is no variant
df['variant_aa'] = [p if v is None else v for p, v in df[['parent_aa', 'variant_aa']].values]
variant_df = df[~df['variant_aa'].isna()] 
variant_df = df[~df['variant_aa'].isna()] # Remove any that didn't have variants
variant_df = variant_df.drop_duplicates(subset=['variant_aa']) # Remove any that didn't have variants

variant_df['variant_id'] = [f'VDEDB{i}' for i in range(0, len(variant_df))]
variant_df['variant_aa_nohis'] = [s.replace('LEHHHHHH', '').replace('HHHHHH', '').replace('*', '') for s in variant_df['variant_aa'].values]
# Save variant DF
variant_df.to_csv('output/protein-evolution-database_V4_proteins_reactions_clean_unique_variants.csv', index=False)

# Hmmm there are too many duplicates (need to check if this is from 
id_col = 'variant_id'
seq_col = 'variant_aa_nohis'

#embedding_df = (variant_df << (EmbedESM(id_col, seq_col, extraction_method='mean', tmp_dir='/disk1/ariane/vscode/DirectedEvolutionDB/analysis/tmpv2/') >> Save('output/protein-evolution-database_V4_embedded_proteins.pkl')))

In [55]:
len(variant_df)

362

# Save embedded file 

Save both only the parents and also the 

In [56]:
import pandas as pd
df.to_csv('output/protein-evolution-database_V5_proteins_reactions_clean.csv', index=False)
parents = df.drop_duplicates(subset=['parent_aa', 'reaction_smiles'])
parents['substrate_smiles'] = [x.split('>')[0] for x in parents['reaction_smiles'].values]
parents = parents[parents['aminoacid_mutations_from_parent'] == '?']
parents = parents.drop_duplicates(subset=['parent_aa', 'substrate_smiles'])
parents.to_csv('output/protein-evolution-database_DF5_proteins_reactions_clean_parents.csv', index=False)

/tmp/ipykernel_1102867/806059250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parents['substrate_smiles'] = [x.split('>')[0] for x in parents['reaction_smiles'].values]


In [57]:
parents

,culture_collection_entry,enzyme_name_from_paper,Uniprot_ID(if applicable),comment,reaction_smiles,parent_DNA_sequence,parent_aminoacid_sequence,aminoacid_mutations_from_parent,variant_DNA_sequence,mutations_from_parent,...,SUBMITTED BY,raw data name,cannonical_reactions,named_reactions,errors,parent_aa,variant_aa,warnings,id,substrate_smiles
0,?,P411-L1,?,3a,O=C1OCCC1=[N+]=[N-].CNC1=CC=CC=C1>>O=C1OCC[C@@...,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,?,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,?,...,?,Asymmetric Carbene NH insertion 2021,CNc1ccccc1.[N-]=[N+]=C1CCOC1=O>>CN(c1ccccc1)[C...,3-diazooxolan-2-one + N-methylaniline =,None,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,None,DEDB0,O=C1OCCC1=[N+]=[N-].CNC1=CC=CC=C1
38,3560,P411BM3-CIS,?,pdb-4H23,O=C(OCC)C=[N+]=[N-].CN1C=CC2=C1C=CC=C2>>CN1C=C...,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,?,?,?,...,Oliver Brandenberg,Indole Amination ACS Cat 2019,CCOC(=O)C=[N+]=[N-].Cn1ccc2ccccc21>>CCOC(=O)Cc...,ethyl 2-diazoacetate + 1-methylindole =,None,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,None,DEDB38,O=C(OCC)C=[N+]=[N-].CN1C=CC2=C1C=CC=C2
58,3560,P411BM3-CIS,?,pdb-4H23,O=C(C=[N+]=[N-])OCC.C12=C(CC=C2)C=CC=C1>>O=C(O...,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,?,?,?,...,Oliver Brandenberg,Indole Amination ACS Cat 2019,C1=Cc2ccccc2C1.CCOC(=O)C=[N+]=[N-]>>CCOC(=O)[C...,ethyl 2-diazoacetate + 1H-indene =,None,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,None,DEDB58,O=C(C=[N+]=[N-])OCC.C12=C(CC=C2)C=CC=C1
61,3560,P411BM3-CIS,?,pdb-4H23,O=C(C=[N+]=[N-])OCC.C1=CC=CCC1>>O=C(OCC)[C@H]1...,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,?,?,?,...,Oliver Brandenberg,Indole Amination ACS Cat 2019,C1=CCCC=C1.CCOC(=O)C=[N+]=[N-]>>CCOC(=O)[C@@H]...,"ethyl 2-diazoacetate + cyclohexa-1,3-diene =",None,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,None,DEDB61,O=C(C=[N+]=[N-])OCC.C1=CC=CCC1
76,4866,P411-C10,?,?,CCC#CC1=CC=CC=C1.O=C(OCC)C=[N+]=[N-]>>[H][C@]2...,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,?,ATGACAATTAAAGAAATGCCTCAGCCAAAAACGTTTGGAGAGCTTA...,?,...,Kai Chen,Internal cyclopropene 2020,CCC#Cc1ccccc1.CCOC(=O)C=[N+]=[N-]>>CCOC(=O)[C@...,but-1-ynylbenzene + ethyl 2-diazoacetate =,None,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,MTIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAP...,None,DEDB76,CCC#CC1=CC=CC=C1.O=C(OCC)C=[N+]=[N-]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,4859,E. coli taurine dioxygenase,P37610,?,C=CC1=CC=CC=C1.O=S(C1=CC=C(C)C=C1)(N=[N+]=[N-]...,ATGAGTGAACGTCTGAGCATTACCCCGCTGGGGCCGTATATCGGCG...,?,?,ATGAGTGAACGTCTGAGCATTACCCCGCTGGGGCCGTATATCGGCG...,?,...,Nathaniel W. Goldberg,NaN,C=Cc1ccccc1.Cc1ccc(S(=O)(=O)N=[N+]=[N-])cc1>>C...,styrene + N-diazo-4-methylbenzenesulfonamide =,None,MSERLSITPLGPYIGAQISGADLTRPLSDNQFEQLYHAVLRHQVVF...,MSERLSITPLGPYIGAQISGADLTRPLSDNQFEQLYHAVLRHQVVF...,None,DEDB1287,C=CC1=CC=CC=C1.O=S(C1=CC=C(C)C=C1)(N=[N+]=[N-])=O
1288,4860,A. thaliana Anthocyanidin synthase,Q96323,?,C=CC1=CC=CC=C1.O=S(C1=CC=C(C)C=C1)(N=[N+]=[N-]...,ATGGTCGCGGTAGAGCGCGTTGAATCCTTAGCGAAGAGCGGAATCA...,?,?,ATGGTCGCGGTAGAGCGCGTTGAATCCTTAGCGAAGAGCGGAATCA...,?,...,Nathaniel W. Goldberg,NaN,C=Cc1ccccc1.Cc1ccc(S(=O)(=O)N=[N+]=[N-])cc1>>C...,styrene + N-diazo-4-methylbenzenesulfonamide =,None,MVAVERVESLAKSGIISIPKEYIRPKEELESINDVFLEEKKEDGPQ...,MVAVERVESLAKSGIISIPKEYIRPKEELESINDVFLEEKKEDGPQ...,None,DEDB1288,C=CC1=CC=CC=C1.O=S(C1=CC=C(C)C=C1)(N=[N+]=[N-])=O
1289,4861,Streptomyces sp. 2-aminobutyric acid chlorinase,D0VX22,?,C=CC1=CC=CC=C1.O=S(C1=CC=C

In [58]:
final_parents = parents.drop_duplicates(subset=['parent_aa', 'paper title'], keep='first')
final_parents.to_csv('parents_helen_thesis.csv')

In [59]:
df.to_csv('output/protein-evolution-database_V5_proteins_reactions_clean.csv', index=False)
first_parent = df.drop_duplicates(subset='parent_aa', keep='first')
# Then also do the last variant with the same chemistry for those ones
chemistries = set(first_parent['reaction_smiles'].values)
canonical_reaction = df[df['reaction_smiles'].isin(chemistries)]
# Get the "most mutated variant"
mutations = [len(aa.split('_')) for aa in canonical_reaction['aminoacid_mutations_from_parent'].values]
canonical_reaction['num_mutations'] = mutations
canonical_reaction = canonical_reaction.sort_values(by='num_mutations', ascending=False)
final_variants = canonical_reaction.drop_duplicates(subset=['parent_aa', 'paper title'], keep='first')
#final_variants

/tmp/ipykernel_1102867/1303138139.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canonical_reaction['num_mutations'] = mutations


In [60]:
final_variants.to_csv('variants_helen_thesis.csv')
